In [0]:
# Caminho no Volume
caminho_volume_csv = "/Volumes/brasileirao2025/bronze/source/brasileirao_2025.csv"

# Criar DataFrame a partir de CSV
df_partidas = spark.read.option("header", True).csv(caminho_volume_csv)

df_partidas.show(5)


In [0]:
import requests, re, json

url = "https://ge.globo.com/futebol/brasileirao-serie-a/"
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text

# Captura o JSON que o site injeta no HTML
match = re.search(r"window\.__INITIAL_STATE__=(.*?);\(function", html)
if match:
    data = json.loads(match.group(1))
    # Dentro desse JSON estão rodadas e jogos
    fases = data.get("championship", {}).get("phases", {})
    for fase_id, fase_info in fases.items():
        rodadas = fase_info.get("rounds", {})
        for rodada_id, rodada_info in rodadas.items():
            jogos = rodada_info.get("matches", [])
            print(f"Rodada {rodada_id} -> {len(jogos)} jogos")
else:
    print("⚠️ Não consegui extrair o JSON do GE")


In [0]:
import requests

url = "https://v3.football.api-sports.io/fixtures"
headers = {"x-apisports-key": "SUA_CHAVE_AQUI"}
params = {"league": 71, "season": 2025}  # 71 = Brasileirão Série A

resp = requests.get(url, headers=headers, params=params)
data = resp.json()

partidas = data.get("response", [])
print(f"Total de partidas: {len(partidas)}")

# Converter para DataFrame Spark
if partidas:
    df_partidas = spark.createDataFrame([
        {
            "id": p["fixture"]["id"],
            "data": p["fixture"]["date"],
            "mandante": p["teams"]["home"]["name"],
            "visitante": p["teams"]["away"]["name"],
            "placar_mandante": p["goals"]["home"],
            "placar_visitante": p["goals"]["away"]
        }
        for p in partidas
    ])
    df_partidas.show(5)


In [0]:
# Instalar bibliotecas necessárias (Databricks Free precisa do pip no notebook)
%pip install requests beautifulsoup4 lxml

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da tabela oficial no site da CBF
url = "https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/2025"

# Fazer request
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Erro ao acessar página da CBF: {response.status_code}")

# Parsear HTML
soup = BeautifulSoup(response.text, "lxml")

# A tabela de jogos está em divs com a classe "swiper-slide"
partidas = []
for jogo in soup.select("div.swiper-slide"):
    rodada = jogo.select_one("h5").get_text(strip=True) if jogo.select_one("h5") else None
    data = jogo.select_one("span.hora-local").get_text(strip=True) if jogo.select_one("span.hora-local") else None
    equipes = jogo.select("strong.time-nome")
    mandante = equipes[0].get_text(strip=True) if len(equipes) > 0 else None
    visitante = equipes[1].get_text(strip=True) if len(equipes) > 1 else None
    estadio = jogo.select_one("span.nome-completo").get_text(strip=True) if jogo.select_one("span.nome-completo") else None
    
    partidas.append({
        "rodada": rodada,
        "data": data,
        "mandante": mandante,
        "visitante": visitante,
        "estadio": estadio
    })

# Criar DataFrame Pandas
df = pd.DataFrame(partidas)

print("Prévia dos dados raspados:")
print(df.head())

# Caminho para salvar no Unity Catalog (Volume Bronze)
caminho_csv = "/Volumes/brasileirao2025/bronze/source/brasileirao_2025.csv"

# Converter para Spark DataFrame
df_spark = spark.createDataFrame(df)

# Salvar como CSV no volume
df_spark.write.mode("overwrite").option("header", True).csv(caminho_csv)

print(f"✅ Dados salvos no volume: {caminho_csv}")

# Criar tabela no catálogo bronze
spark.sql(f"""
CREATE TABLE IF NOT EXISTS brasileirao2025.bronze.tb_bronze_partidas
USING csv
OPTIONS (header = "true")
LOCATION '{caminho_csv}'
""")

print("✅ Tabela criada no Unity Catalog: brasileirao2025.bronze.tb_bronze_partidas")

# Conferir dados
spark.sql("SELECT * FROM brasileirao2025.bronze.tb_bronze_partidas LIMIT 10").show(truncate=False)
